In [1]:
import numpy as np
import pandas as pd
import mlflow
import hashlib
from mlflow.tracking import MlflowClient
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

**Sample data**

In [37]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [39]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [41]:
# Save locally for artifact logging
train_df = pd.DataFrame(X_train, columns=[f"f{i}" for i in range(X_train.shape[1])])
train_df["target"] = y_train
train_path = "train_dataset.csv"
train_df.to_csv(train_path, index=False)

In [43]:
def hash_array(arr):
    """Compute a hash for a numpy array"""
    return hashlib.md5(arr.tobytes()).hexdigest()

In [45]:
data_hash = hash_array(X_train)

In [47]:
data_hash

'aedcfb766e8cc9b6a45fdd190f4ba932'

**Model training and registering to MLFlow**

**Model 1: Logistic Regression**

In [53]:
params = {
"solver": "lbfgs",
"max_iter": 100000,
"multi_class": "auto",
"random_state":42
}

lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)


In [55]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [57]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [59]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

In [61]:
mlflow.set_experiment("First experiment")

<Experiment: artifact_location='mlflow-artifacts:/758964068121251249', creation_time=1767183284858, experiment_id='758964068121251249', last_update_time=1767183284858, lifecycle_stage='active', name='First experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [63]:
with mlflow.start_run(run_name = "logistic_regression_v3"):
    mlflow.log_params(params)
    mlflow.log_metrics(
        {
            "accuracy": report_dict["accuracy"],
            "recall_class_0": report_dict["0"]["recall"],
            "recall_class_1": report_dict["1"]["recall"],
            "precision_class_0": report_dict["0"]["precision"],
            "precision_class_1": report_dict["1"]["precision"],
            "f1_score_class_0": report_dict["0"]["f1-score"],
            "f1_score_class_1": report_dict["1"]["f1-score"],
            
        }
    )

    mlflow.set_tag("data_hash", data_hash)
    mlflow.log_artifact(train_path)
    
    mlflow.sklearn.log_model(lr, 
                             name = "model",
                             registered_model_name = "Logistic Regression")

2026/01/02 11:55:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Logistic Regression' already exists. Creating a new version of this model...
2026/01/02 11:55:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression, version 3
Created version '3' of model 'Logistic Regression'.


🏃 View run logistic_regression_v3 at: http://127.0.0.1:5000/#/experiments/758964068121251249/runs/40efef377ee4449b80f5ab1fca2e0b83
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758964068121251249


**Model 2: Random Forest**

In [61]:
params2 = {
    "n_estimators": 20,
    "max_depth":5
    
}

rf = RandomForestClassifier(**params2)
rf.fit(X_train, y_train)

y_pred2 = rf.predict(X_test)

In [63]:
report2 = classification_report(y_test, y_pred2)
print(report2)

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       270
           1       0.96      0.77      0.85        30

    accuracy                           0.97       300
   macro avg       0.97      0.88      0.92       300
weighted avg       0.97      0.97      0.97       300



In [65]:
report2 = classification_report(y_test, y_pred2, output_dict = True)
report2

{'0': {'precision': 0.9746376811594203,
  'recall': 0.9962962962962963,
  'f1-score': 0.9853479853479854,
  'support': 270.0},
 '1': {'precision': 0.9583333333333334,
  'recall': 0.7666666666666667,
  'f1-score': 0.8518518518518519,
  'support': 30.0},
 'accuracy': 0.9733333333333334,
 'macro avg': {'precision': 0.9664855072463768,
  'recall': 0.8814814814814815,
  'f1-score': 0.9185999185999186,
  'support': 300.0},
 'weighted avg': {'precision': 0.9730072463768117,
  'recall': 0.9733333333333334,
  'f1-score': 0.9719983719983721,
  'support': 300.0}}

In [67]:
with mlflow.start_run(run_name = "random_forest_v1"):
    mlflow.log_params(params2)
    mlflow.log_metrics(
        {
            "accuracy": report2["accuracy"],
            "recall_class_0": report2["0"]["recall"],
            "recall_class_1": report2["1"]["recall"],
            "precision_class_0": report2["0"]["precision"],
            "precision_class_1": report2["1"]["precision"],
            "f1_score_class_0": report2["0"]["f1-score"],
            "f1_score_class_1": report2["1"]["f1-score"],
            
        }
    )

    mlflow.set_tag("data_hash", data_hash)
    mlflow.log_artifact(train_path)
    
    mlflow.sklearn.log_model(rf, 
                             name = "model",
                              registered_model_name = "Random Forest")

2025/12/31 17:51:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Random Forest'.
2025/12/31 17:51:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest, version 1
Created version '1' of model 'Random Forest'.


🏃 View run random_forest_v1 at: http://127.0.0.1:5000/#/experiments/758964068121251249/runs/f72bc8416d1d4fe89fd5b86053818cd6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/758964068121251249


**Fetch the model through MLFlow client and predict**

In [10]:
client = MlflowClient()
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1767183328638, deployment_job_id=None, deployment_job_state=None, description='', last_updated_timestamp=1767242868246, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1767241296962, current_stage='None', deployment_job_state=None, description='', last_updated_timestamp=1767241296962, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='accuracy', model_id='m-02de08a5ad104ed093ae0ebf22a516af', run_id='bc0c635f59af43fba25f9cfcfbe54bc0', step=0, timestamp=1767241273928, value=0.928>,
  <Metric: dataset_digest=None, dataset_name=None, key='f1_score_class_0', model_id='m-02de08a5ad104ed093ae0ebf22a516af', run_id='bc0c635f59af43fba25f9cfcfbe54bc0', step=0, timestamp=1767241273928, value=0.960352422907489>,
  <Metric: dataset_digest=None, dataset_name=None, key='f1_score_class_1', model_id='m-02de08a5ad104ed093ae0ebf22a516af', run_id='bc0c635f59af43fba25f9cfcfbe54bc0', step=0, timestamp=1767241273928, value=0.60869565

In [80]:
model_name = "Random Forest"
latest_versions = client.get_latest_versions(name = model_name, stages = None)
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1767183677129, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1767183677129, metrics=None, model_id=None, name='Random Forest', params=None, run_id='f72bc8416d1d4fe89fd5b86053818cd6', run_link='', source='models:/m-76e633b0a5d44c0ca1ff6afee2b9cb92', status='READY', status_message=None, tags={}, user_id='', version='1'>]

In [82]:
latest_version = latest_versions[0].version
latest_version

'1'

In [36]:
loaded_model_uri = f"models:/{model_name}/{latest_version}"
loaded_rf = mlflow.sklearn.load_model(loaded_model_uri)

In [75]:
ypred_load = loaded_rf.predict(X_test)
ypred_load

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0])

In [31]:
model_name = "Logistic Regression"
latest_versions = client.get_latest_versions(name = model_name, stages = None)
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1767241296962, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1767241296962, metrics=None, model_id=None, name='Logistic Regression', params=None, run_id='bc0c635f59af43fba25f9cfcfbe54bc0', run_link='', source='models:/m-02de08a5ad104ed093ae0ebf22a516af', status='READY', status_message=None, tags={}, user_id='', version='2'>]

**Link the data version to the model version**

In [65]:
client.set_model_version_tag(
    name = "Logistic Regression",
    version = "3",
    key = "data_hash",
    value = data_hash
)

In [67]:
mv = client.get_model_version(
    name="Logistic Regression",
    version=3
)

dataset_hash = mv.tags["data_hash"]
dataset_hash

'aedcfb766e8cc9b6a45fdd190f4ba932'

In [59]:
runs = client.search_runs(
    experiment_ids=["758964068121251249"],
    filter_string=f"tags.data_hash = '{dataset_hash}'"
)

run_id = runs[0].info.run_id
run_id

'f72bc8416d1d4fe89fd5b86053818cd6'

In [61]:
local_path = client.download_artifacts(
    run_id = run_id,
    path = "train_dataset.csv"
)

df = pd.read_csv(local_path)
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,target
0,1.186738,1.511441,0.784904,-0.700854,-0.244227,-0.733189,1.268879,-0.612295,-0.138303,-0.247534,0
1,-1.288103,-1.038553,-2.070921,0.032390,0.738601,1.379845,-1.202812,0.496070,-1.503770,0.624742,0
2,1.663938,1.551421,2.250242,-0.295726,-0.789045,-1.578862,1.614572,-0.699556,1.366006,-0.682905,1
3,-0.865648,-0.891806,-0.999171,0.256317,0.343875,0.739221,-0.864509,0.387647,-0.477969,0.305181,0
4,-1.570485,-1.276852,-2.503404,0.052343,0.892164,1.672033,-1.469575,0.607794,-1.804245,0.755416,0
...,...,...,...,...,...,...,...,...,...,...,...
695,-1.947827,-1.536446,-3.200468,0.007818,1.143649,2.119563,-1.808995,0.740617,-2.367419,0.964837,0
696,0.238268,0.639499,-0.522861,-0.547278,0.215287,0.178809,0.352139,-0.217006,-0.950992,0.149062,0
697,0.431016,0.900136,-0.426061,-0.679442,0.187539,0.074426,0.562619,-0.320696,-1.015085,0.117820,0
698,0.798399,1.500347,-0.450989,-1.056484,0.216005,-0.024188,0.993772,-0.547286,-1.421401,0.119449,0


In [84]:
client.set_model_version_tag(
    name = "Random Forest",
    version = "1",
    key = "data_hash",
    value = data_hash
)


mv = client.get_model_version(
    name="Random Forest",
    version=1
)

dataset_hash = mv.tags["data_hash"]
dataset_hash

'aedcfb766e8cc9b6a45fdd190f4ba932'

**Push best model in production**

In [12]:
candidate_models = [
    "Logistic Regression",
    "Random Forest"
]

In [19]:
best_model = None
best_version = None
best_accuracy = -1

for model_name in candidate_models:
    versions = client.get_latest_versions(model_name)

    for v in versions:
        run = client.get_run(v.run_id)
        accuracy = run.data.metrics.get("accuracy")

        if accuracy is not None and accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model_name
            best_version = v.version

print(f"Best model: {best_model}")
print(f"Best accuracy: {best_accuracy}")
print(f"Best version: {best_version}")

Best model: Random Forest
Best accuracy: 0.9733333333333334
Best version: 1


In [21]:
client.set_registered_model_alias(
    name=best_model,
    alias="prod",
    version=best_version
)


In [23]:
prod_version = client.get_model_version_by_alias(
    name="Random Forest",
    alias="prod"
)
prod_version

<ModelVersion: aliases=['prod'], creation_timestamp=1767183677129, current_stage='None', deployment_job_state=None, description='', last_updated_timestamp=1767183677129, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='accuracy', model_id='m-76e633b0a5d44c0ca1ff6afee2b9cb92', run_id='f72bc8416d1d4fe89fd5b86053818cd6', step=0, timestamp=1767183667062, value=0.9733333333333334>,
 <Metric: dataset_digest=None, dataset_name=None, key='f1_score_class_0', model_id='m-76e633b0a5d44c0ca1ff6afee2b9cb92', run_id='f72bc8416d1d4fe89fd5b86053818cd6', step=0, timestamp=1767183667062, value=0.9853479853479854>,
 <Metric: dataset_digest=None, dataset_name=None, key='f1_score_class_1', model_id='m-76e633b0a5d44c0ca1ff6afee2b9cb92', run_id='f72bc8416d1d4fe89fd5b86053818cd6', step=0, timestamp=1767183667062, value=0.8518518518518519>,
 <Metric: dataset_digest=None, dataset_name=None, key='precision_class_0', model_id='m-76e633b0a5d44c0ca1ff6afee2b9cb92', run_id='f72bc8416d1d4fe89fd5b8605381

In [35]:
model = mlflow.sklearn.load_model(
    model_uri="models:/Random Forest@prod"
)
model

RandomForestClassifier(max_depth=5, n_estimators=20)